Adding an empty row and some words so pandas reads it as label

In [46]:
#Adding an empty row and some words so pandas reads it as labels
import csv

# Specify the path to your input CSV file
input_csv = 'radar/20-20-15.139673_896551.csv'

# Specify the path to your output CSV file
output_csv = 'output1.csv'

# Read the existing data from the input CSV file
with open(input_csv, 'r') as file:
    reader = csv.reader(file)
    rows = list(reader)

# Insert an empty row at the beginning of the data
rows.insert(0, [])

# Add 'Unnamed' to the entire empty row
rows[0] = ['Unnamed'] * len(rows[1])

# Write the updated data to the output CSV file
with open(output_csv, 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(rows)

CSV file updated successfully.


Deleting the first column (labels)

In [48]:

#Deleting the first column (labels)
import csv

# Specify the path to your input CSV file
input_csv = 'output1.csv'

# Specify the path to your output CSV file
output_csv = 'output2.csv'

# Read the existing data from the input CSV file
with open(input_csv, 'r') as file:
    reader = csv.reader(file)
    rows = list(reader)

# Remove the first column from each row
for row in rows:
    del row[0]

# Write the updated data to the output CSV file
with open(output_csv, 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(rows)

Adding rows for x, y and z values

In [17]:
#Adding rows for x, y and z values
import numpy as np
import pandas as pd

df = pd.read_csv('output2.csv')
# Clean and convert the values in the 'Range' and 'Azimuth' rows to numeric types
range_vals = pd.to_numeric(df.iloc[0], errors='coerce')
azimuth_vals = pd.to_numeric(df.iloc[1], errors='coerce')

# Calculate the value of 'x' for each corresponding radar value
x_values = range_vals * np.cos(np.deg2rad(azimuth_vals))
y_values = range_vals * np.sin(np.deg2rad(azimuth_vals))
z_values = np.zeros(len(range_vals))

# Insert the calculated 'x' values at the 8th row
df = pd.concat([df.iloc[:0], pd.DataFrame([x_values], columns=df.columns), df.iloc[0:]], ignore_index=True)
df = pd.concat([df.iloc[:1], pd.DataFrame([y_values], columns=df.columns), df.iloc[1:]], ignore_index=True)
df = pd.concat([df.iloc[:2], pd.DataFrame([z_values], columns=df.columns), df.iloc[2:]], ignore_index=True)
# df = pd.concat([df.iloc[:0], pd.DataFrame([x_values], columns=df.columns), df.iloc[0:]], ignore_index=True)
df.to_csv("final1.csv", index=False)

Automating this process

In [16]:
import os
import csv
import numpy as np
import pandas as pd

# Specify the input folder containing CSV files
input_folder = 'iith-to-ptrpally'

# Specify the output folder to save the modified CSV files
output_folder = 'iith-to-ptrpally-xyz'

# Create the output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

# Process each file in the input folder
for filename in os.listdir(input_folder):
    if filename.endswith('.csv'):
        # Construct the paths for input and output files
        input_csv = os.path.join(input_folder, filename)
        output_csv = os.path.join(output_folder, filename)
        
        # Step 1: Adding an empty row and 'Unnamed' labels
        with open(input_csv, 'r') as file:
            reader = csv.reader(file)
            rows = list(reader)
        
        rows.insert(0, [])
        rows[0] = ['Unnamed'] * len(rows[1])
        
        with open(output_csv, 'w', newline='') as file:
            writer = csv.writer(file)
            writer.writerows(rows)
        
        # Step 2: Deleting the first column (labels)
        with open(output_csv, 'r') as file:
            reader = csv.reader(file)
            rows = list(reader)
        
        for row in rows:
            del row[0]
        
        with open(output_csv, 'w', newline='') as file:
            writer = csv.writer(file)
            writer.writerows(rows)
        
        # Step 3: Adding rows for x, y, and z values
        df = pd.read_csv(output_csv)
        range_vals = pd.to_numeric(df.iloc[0], errors='coerce')
        azimuth_vals = pd.to_numeric(df.iloc[1], errors='coerce')
        
        x_values = range_vals * np.cos(azimuth_vals)
        y_values = range_vals * np.sin(azimuth_vals)
        z_values = np.zeros(len(range_vals))
        
        df = pd.concat([df.iloc[:0], pd.DataFrame([x_values], columns=df.columns), df.iloc[0:]], ignore_index=True)
        df = pd.concat([df.iloc[:1], pd.DataFrame([y_values], columns=df.columns), df.iloc[1:]], ignore_index=True)
        df = pd.concat([df.iloc[:2], pd.DataFrame([z_values], columns=df.columns), df.iloc[2:]], ignore_index=True)
        
        df.to_csv(output_csv, index=False)